In [1]:
#Caesar Cipher
def encrypt(text,s):
    result = ""
 
    # traverse text
    for i in range(len(text)):
        char = text[i]
 
        # Encrypt uppercase characters
        if (char.isupper()):
            result += chr((ord(char) + s-65) % 26 + 65)
 
        # Encrypt lowercase characters
        else:
            result += chr((ord(char) + s - 97) % 26 + 97)
 
    return result
 
#check the above function
text = "ATTACKATONCE"
s = 4
print ("Text  : " + text)
print ("Shift : " + str(s))
print ("Cipher: " + encrypt(text,s))

Text  : ATTACKATONCE
Shift : 4
Cipher: EXXEGOEXSRGI


In [7]:
#PlayFair Cipher
def create_matrix(key):
    key = key.upper()
    matrix = [[0 for i in range (5)] for j in range(5)]
    letters_added = []
    row = 0
    col = 0
    # add the key to the matrix
    for letter in key:
        if letter not in letters_added:
            matrix[row][col] = letter
            letters_added.append(letter)
        else:
            continue
        if (col==4):
            col = 0
            row += 1
        else:
            col += 1
    #Add the rest of the alphabet to the matrix
    # A=65 ... Z=90
    for letter in range(65,91):
        if letter==74: # I/J are in the same position
                continue
        if chr(letter) not in letters_added: # Do not add repeated letters
            letters_added.append(chr(letter))
            
    #print (len(letters_added), letters_added)
    index = 0
    for i in range(5):
        for j in range(5):
            matrix[i][j] = letters_added[index]
            index+=1
    return matrix

def separate_same_letters(message):
    index = 0
    while (index<len(message)):
        l1 = message[index]
        if index == len(message)-1:
            message = message + 'X'
            index += 2
            continue
        l2 = message[index+1]
        if l1==l2:
            message = message[:index+1] + "X" + message[index+1:]
        index +=2   
    return message

def indexOf(letter,matrix):
    for i in range (5):
        try:
            index = matrix[i].index(letter)
            return (i,index)
        except:
            continue
#Implementation of the playfair cipher
#If encrypt=True the method will encrypt the message
# otherwise the method will decrypt
def playfair(key, message, encrypt=True):
    inc = 1
    if encrypt==False:
        inc = -1
    matrix = create_matrix(key)
    message = message.upper()
    message = message.replace(' ','')    
    message = separate_same_letters(message)
    cipher_text=''
    for (l1, l2) in zip(message[0::2], message[1::2]):
        row1,col1 = indexOf(l1,matrix)
        row2,col2 = indexOf(l2,matrix)
        if row1==row2: #Rule 2, the letters are in the same row
            cipher_text += matrix[row1][(col1+inc)%5] + matrix[row2][(col2+inc)%5]
        elif col1==col2:# Rule 3, the letters are in the same column
            cipher_text += matrix[(row1+inc)%5][col1] + matrix[(row2+inc)%5][col2]
        else: #Rule 4, the letters are in a different row and column
            cipher_text += matrix[row1][col2] + matrix[row2][col1]
    
    return cipher_text
if __name__=='__main__':
    # a sample of encryption and decryption
    print ('Encripting')
    inp = playfair('secret', 'instruments')
    print ( inp)
    print ('Decrypting')
    print ( playfair('secret', inp, False))

Encripting
HOESTQITUSCV
Decrypting
INSTRUMENTSX


In [11]:
#Hill Cipher

import numpy
 
def create_matrix_from(key):
    m=[[0] * 3 for i in range(3)]
    for i in range(3):
        for j in range(3):
            m[i][j] = ord(key[3*i+j]) % 65
    return m
# C = P*K mod 26
def encrypt(P, K):
    C=[0,0,0]
    C[0] = (K[0][0]*P[0] + K[1][0]*P[1] + K[2][0]*P[2]) % 26
    C[1] = (K[0][1]*P[0] + K[1][1]*P[1] + K[2][1]*P[2]) % 26
    C[2] = (K[0][2]*P[0] + K[1][2]*P[1] + K[2][2]*P[2]) % 26
    return C

def Hill(message, K):
    cipher_text = []
    #Transform the message 3 characters at a time
    for i in range(0,len(message), 3):
        P=[0, 0, 0]
        #Assign the corresponfing integer value to each letter
        for j in range(3):
            P[j] = ord(message[i+j]) % 65
        #Encript three letters
        C = encrypt(P,K)
        #Add the encripted 3 letters to the final cipher text
        for j in range(3):
            cipher_text.append(chr(C[j] + 65))
        #Repeat until all sets of three letters are processed.
        
    #return a string
    return "".join(cipher_text)

def MatrixInverse(K):
    det = int(numpy.linalg.det(K))
    det_multiplicative_inverse = pow(det, -1, 26)
    K_inv = [[0] * 3 for i in range(3)]
    for i in range(3):
        for j in range(3):
            Dji = K
            Dji = numpy.delete(Dji, (j), axis=0)
            Dji = numpy.delete(Dji, (i), axis=1)
            det = Dji[0][0]*Dji[1][1] - Dji[0][1]*Dji[1][0]
            K_inv[i][j] = (det_multiplicative_inverse * pow(-1,i+j) * det) % 26
    return K_inv

if __name__ == "__main__":
    
    message = "MYSECRETMESSAGE"
    key = "RRFVSVCCT"
    #Create the matrix K that will be used as the key
    K = create_matrix_from(key)
    print(K)
    # C = P * K mod 26
    cipher_text = Hill(message, K)
    print ('Cipher text: ', cipher_text)
    
    
    # Decrypt
    # P = C * K^-1 mod 26
    K_inv = MatrixInverse(K)            
    plain_text = Hill(cipher_text, K_inv)
    print ('Plain text: ', plain_text)

[[17, 17, 5], [21, 18, 21], [2, 2, 19]]
Cipher text:  QWWOIVXSXOMMEMU
Plain text:  MYSECRETMESSAGE


In [12]:
#Railfence Cipher
def encryptRailFence(text, key):
 
    # create the matrix to cipher
    # plain text key = rows ,
    # length(text) = columns
    # filling the rail matrix
    # to distinguish filled
    # spaces from blank ones
    rail = [['\n' for i in range(len(text))]
                for j in range(key)]
     
    # to find the direction
    dir_down = False
    row, col = 0, 0
     
    for i in range(len(text)):
         
        # check the direction of flow
        # reverse the direction if we've just
        # filled the top or bottom rail
        if (row == 0) or (row == key - 1):
            dir_down = not dir_down
         
        # fill the corresponding alphabet
        rail[row][col] = text[i]
        col += 1
         
        # find the next row using
        # direction flag
        if dir_down:
            row += 1
        else:
            row -= 1
    # now we can construct the cipher
    # using the rail matrix
    result = []
    for i in range(key):
        for j in range(len(text)):
            if rail[i][j] != '\n':
                result.append(rail[i][j])
    return("" . join(result))
     
# This function receives cipher-text
# and key and returns the original
# text after decryption
def decryptRailFence(cipher, key):
 
    # create the matrix to cipher
    # plain text key = rows ,
    # length(text) = columns
    # filling the rail matrix to
    # distinguish filled spaces
    # from blank ones
    rail = [['\n' for i in range(len(cipher))]
                for j in range(key)]
     
    # to find the direction
    dir_down = None
    row, col = 0, 0
     
    # mark the places with '*'
    for i in range(len(cipher)):
        if row == 0:
            dir_down = True
        if row == key - 1:
            dir_down = False
         
        # place the marker
        rail[row][col] = '*'
        col += 1
         
        # find the next row
        # using direction flag
        if dir_down:
            row += 1
        else:
            row -= 1
             
    # now we can construct the
    # fill the rail matrix
    index = 0
    for i in range(key):
        for j in range(len(cipher)):
            if ((rail[i][j] == '*') and
            (index < len(cipher))):
                rail[i][j] = cipher[index]
                index += 1
         
    # now read the matrix in
    # zig-zag manner to construct
    # the resultant text
    result = []
    row, col = 0, 0
    for i in range(len(cipher)):
         
        # check the direction of flow
        if row == 0:
            dir_down = True
        if row == key-1:
            dir_down = False
             
        # place the marker
        if (rail[row][col] != '*'):
            result.append(rail[row][col])
            col += 1
             
        # find the next row using
        # direction flag
        if dir_down:
            row += 1
        else:
            row -= 1
    return("".join(result))
 
# Driver code
if __name__ == "__main__":
    print(encryptRailFence("attack at once", 2))
    print(encryptRailFence("GeeksforGeeks ", 3))
    print(encryptRailFence("defend the east wall", 3))
     
    # Now decryption of the
    # same cipher-text
    print(decryptRailFence("GsGsekfrek eoe", 3))
    print(decryptRailFence("atc toctaka ne", 2))
    print(decryptRailFence("dnhaweedtees alf  tl", 3))

atc toctaka ne
GsGsekfrek eoe
dnhaweedtees alf  tl
GeeksforGeeks 
attack at once
defend the east wall


In [1]:
#RSA Algorithm
import math
 
 
def gcd(a, h):
    temp = 0
    while(1):
        temp = a % h
        if (temp == 0):
            return h
        a = h
        h = temp
 
 
p = 3
q = 7
n = p*q
e = 2
phi = (p-1)*(q-1)
 
while (e < phi):
 
    # e must be co-prime to phi and
    # smaller than phi.
    if(gcd(e, phi) == 1):
        break
    else:
        e = e+1
 
# Private key (d stands for decrypt)
# choosing d such that it satisfies
# d*e = 1 + k * totient
 
k = 2
d = (1 + (k*phi))/e
 
# Message to be encrypted
msg = 12.0
 
print("Message data = ", msg)
 
# Encryption c = (msg ^ e) % n
c = pow(msg, e)
c = math.fmod(c, n)
print("Encrypted data = ", c)
 
# Decryption m = (c ^ d) % n
m = pow(c, d)
m = math.fmod(m, n)
print("Original Message Sent = ", m)

Message data =  12.0
Encrypted data =  3.0
Original Message Sent =  12.0


In [9]:
#Diffie Hellman
from random import randint

p = int(input("Enter private key of Alice(p):"))
g = int(input("Enter private key of Bob(g):"))
a = randint(1, 100)      # syntax reference: https://docs.python.org/3/library/random.html
b = randint(1, 100)
A = pow(g, a, p)          # syntax reference: https://www.geeksforgeeks.org/pow-in-python/
B = pow(g, b, p)
Ka = pow(B, a, p)
Kb = pow(A, b, p)
print("Secret key for Alice is = ", str(Ka))
print("Secret key for Bob is = ", str(Kb))

Enter private key of Alice(p):23
Enter private key of Bob(g):9
Secret key for Alice is =  8
Secret key for Bob is =  8


In [11]:
#MD5
import hashlib 
 
result = hashlib.md5(b'PlainString') 
print("Hash Value : ", end ="")
print(result)
print("Equivalent Byte : ", end ="") 
print(result.digest())
print("Hexadecimal Equivalent : ", end ="") 
print(result.hexdigest()) 

Hash Value : <md5 _hashlib.HASH object @ 0x000002105B280E10>
Equivalent Byte : b'\x8e\x90\x1d\xdfcb\xcd\xbc\xb4\xeea\xcb\xc6\rJ\x92'
Hexadecimal Equivalent : 8e901ddf6362cdbcb4ee61cbc60d4a92


In [14]:
#Elliptic Curve Cryptosystem
!pip install tinyec
from tinyec import registry
import secrets
  
# Function to calculate compress point 
# of elliptic curves
def compress(publicKey):
 return hex(publicKey.x) + hex(publicKey.y % 2)[2:]
  
# The elliptic curve which is used for the ECDH calculations
curve = registry.get_curve('brainpoolP256r1')
  
# Generation of secret key and public key
Ka = secrets.randbelow(curve.field.n)
X = Ka * curve.g 
print("X:", compress(X))
Kb = secrets.randbelow(curve.field.n)
Y = Kb * curve.g 
print("Y:", compress(Y))
print("Currently exchange the publickey (e.g. through Internet)")
  
# (A_SharedKey): represents user A
# (B_SharedKey): represents user B
A_SharedKey = Ka * Y
print("A shared key :",compress(A_SharedKey))
B_SharedKey = Kb * X
print("(B) shared key :",compress(B_SharedKey))
print("Equal shared keys:", A_SharedKey == B_SharedKey)

X: 0x2c89b4025756e59143949bfdc7911220ecf8c9bfda4afd521e0678640d798f861
Y: 0xa3d3035d5769b482a1d99ef30527c110f000f30771397d2289687e5fcfc2285b1
Currently exchange the publickey (e.g. through Internet)
A shared key : 0x4ea20e2e169b306e466a2e40b320f3f005258cc2531e44486e1cdcf25cf29e121
(B) shared key : 0x4ea20e2e169b306e466a2e40b320f3f005258cc2531e44486e1cdcf25cf29e121
Equal shared keys: True


In [15]:
#ELGamal Cryptosystem
import random
from math import pow
 
a = random.randint(2, 10)
 
def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b;
    else:
        return gcd(b, a % b)
 
# Generating large random numbers
def gen_key(q):
 
    key = random.randint(pow(10, 20), q)
    while gcd(q, key) != 1:
        key = random.randint(pow(10, 20), q)
 
    return key
 
# Modular exponentiation
def power(a, b, c):
    x = 1
    y = a
 
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c;
        y = (y * y) % c
        b = int(b / 2)
 
    return x % c
 
# Asymmetric encryption
def encrypt(msg, q, h, g):
 
    en_msg = []
 
    k = gen_key(q)# Private key for sender
    s = power(h, k, q)
    p = power(g, k, q)
     
    for i in range(0, len(msg)):
        en_msg.append(msg[i])
 
    print("g^k used : ", p)
    print("g^ak used : ", s)
    for i in range(0, len(en_msg)):
        en_msg[i] = s * ord(en_msg[i])
 
    return en_msg, p
 
def decrypt(en_msg, p, key, q):
 
    dr_msg = []
    h = power(p, key, q)
    for i in range(0, len(en_msg)):
        dr_msg.append(chr(int(en_msg[i]/h)))
         
    return dr_msg
 
# Driver code
def main():
 
    msg = 'encryption'
    print("Original Message :", msg)
 
    q = random.randint(pow(10, 20), pow(10, 50))
    g = random.randint(2, q)
 
    key = gen_key(q)# Private key for receiver
    h = power(g, key, q)
    print("g used : ", g)
    print("g^a used : ", h)
 
    en_msg, p = encrypt(msg, q, h, g)
    dr_msg = decrypt(en_msg, p, key, q)
    dmsg = ''.join(dr_msg)
    print("Decrypted Message :", dmsg);
 
 
if __name__ == '__main__':
    main()

Original Message : encryption
g used :  52684573907222683847665517330908798702244411791357
g^a used :  27096482127321403299371565695602851610462891229767
g^k used :  61593590289583019273979780728959451475577481081515
g^ak used :  26968669868425413136100416079555116676633002852515
Decrypted Message : encryption


In [ ]:
#Chinese Remainder Theorem
def inv(a, m) : 
    m0 = m 
    x0 = 0
    x1 = 1
    if (m == 1) : 
        return 0
    while (a > 1) : 
        q = a // m 
        t = m 
        m = a % m 
        a = t 
        t = x0 
        x0 = x1 - q * x0 
        x1 = t 
    if (x1 < 0) : 
        x1 = x1 + m0 
  
    return x1 
  
def findMinX(num, rem, k) : 
    prod = 1
    for i in range(0, k) : 
        prod = prod * num[i] 

    result = 0
   
    for i in range(0,k): 
        pp = prod // num[i] 
        result = result + rem[i] * inv(pp, num[i]) * pp 
      
      
    return result % prod 
  
# Driver method 
num = [3, 4, 5] 
rem = [2, 3, 1] 
k = len(num) 
print( "x is " , findMinX(num, rem, k)) 